## Finetuning DistilBERT using SQuAD 2.0

This notebook contains the following fine-tuning groups:  Model_13 and Model_16

....

The starting-point for code in this file was found in the Medium blog post titled, Question Answering with DistilBERT (https://medium.com/@sabrinaherbst/question-answering-with-distilbert-ba3e178fdf3d). Main differences include:
 - The DistilBERT model was pre-trined using SQuAD 2.0, rather than SQuAD 1.0
 - Exploring traditional split that included unseen test data  (i.e. not validation data)
 - Addition of adhoc dropout rate setting

....

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load libraries

from transformers import DistilBertModel, DistilBertForMaskedLM, DistilBertConfig, \
            DistilBertTokenizerFast, AutoTokenizer, BertModel, BertForMaskedLM, BertTokenizerFast, BertConfig
from torch import nn
from pathlib import Path
import torch
import pandas as pd
from typing import Optional
from tqdm.auto import tqdm
from torch.optim import AdamW, RMSprop
import numpy as np

import sys
sys.path.append('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project')
from qa_model import QuestionDistilBERT, SimpleQuestionDistilBERT, ReuseQuestionDistilBERT, Dataset, test_model
from util import eval_test_set, count_parameters, print_test_set_incorrect_predictions, \
            analyze_test_set_statistics, analyze_test_set_performance

# Load tokenizer


In [ ]:
# Load DistilBERT tokenizer, use uncased (lowercase) vocabulary

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Load data_2 (Traditional split)

Note: Dataset loader is the "Variable-Length Trunc" described in the report

In [ ]:
# Get paths for all SQuAD dataset text files in training directory -- data_2, Traditional Split
squad_paths_2 = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')]

# Create full training dataset
dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')],
                  natural_question_paths=None,
                  hotpotqa_paths=None,
                  tokenizer=tokenizer)

# Create a half size training dataset
full_size = len(dataset_2)
half_size = full_size // 2  # Integer division to get exact half
subset_indices = torch.randperm(full_size)[:half_size]
dataset_2_half = torch.utils.data.Subset(dataset_2, subset_indices)
loader_2_half = torch.utils.data.DataLoader(dataset_2_half, batch_size=8)
loader_2 = torch.utils.data.DataLoader(dataset_2, batch_size=8)

# Print sizes for verification
print(f"Original Training Dataset Size: {full_size}")
print(f"Half Training Dataset Size: {len(dataset_2_half)}")

## load the validation dataset -- used to be labeled as "test", test_dataset changed to val_dataset, test_loader changed to val_loader
val_dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/validation_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
val_loader_2 = torch.utils.data.DataLoader(val_dataset_2, batch_size=4)
print(f"Approximate Validation Dataset Size: {len(val_dataset_2)}")

## load the test dataset -- test_dataset and test_loader should not be used during training
test_dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/test_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
test_loader_2 = torch.utils.data.DataLoader(test_dataset_2, batch_size=4)
print(f"Approximate Test Dataset Size: {len(test_dataset_2)}")

Original Training Dataset Size: 113000
Half Training Dataset Size: 56500
Approximate Validation Dataset Size: 14000
Approximate Test Dataset Size: 14000


# Model_13
Train DistilBERT using SQuAD 2.0 --- data_2, Traditional Split; Alter training parameters to 4 epochs, dropout rate of 0.225 with AdamW optimizer rather than RMSProp

In [ ]:
# Load pre-trained DistilBERT model for masked language modeling
model_13 = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# Get model configuration (architecture, hyperparameters)
config_13 = DistilBertConfig.from_pretrained("distilbert-base-uncased")

# Extract base DistilBERT model without MLM head
mod_13 = model_13.distilbert

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_13 = SimpleQuestionDistilBERT(mod_13)
model_13.set_dropout_rate(0.225)
model_13.to(device)

# Verify the dropout rates for each layer
for name, module in model_13.named_modules():
    if isinstance(module, nn.Dropout):
        print(f"{name}: {module.p}")

set_dropout_rate
distilbert.embeddings.dropout: 0.225
distilbert.transformer.layer.0.attention.dropout: 0.225
distilbert.transformer.layer.0.ffn.dropout: 0.225
distilbert.transformer.layer.1.attention.dropout: 0.225
distilbert.transformer.layer.1.ffn.dropout: 0.225
distilbert.transformer.layer.2.attention.dropout: 0.225
distilbert.transformer.layer.2.ffn.dropout: 0.225
distilbert.transformer.layer.3.attention.dropout: 0.225
distilbert.transformer.layer.3.ffn.dropout: 0.225
distilbert.transformer.layer.4.attention.dropout: 0.225
distilbert.transformer.layer.4.ffn.dropout: 0.225
distilbert.transformer.layer.5.attention.dropout: 0.225
distilbert.transformer.layer.5.ffn.dropout: 0.225
dropout: 0.225


In [ ]:
# Set model to training mode (enables dropout, batch norm, etc.)
model_13.train()

# Initialize RMSprop optimizer and learning rate of 4e-5
optim = AdamW(model_13.parameters(), lr=4e-5)

In [ ]:
# Train the DistilBERT model with data_2 for 4 epochs
# Validation data used to evaluate performance during training

epochs = 4

for epoch in range(epochs):
   # Training loop
   loop = tqdm(loader_2, leave=True)  # Progress bar for training batches
   model_13.train()  # Set model to training mode
   mean_training_loss = []

   for batch in loop:
       # Zero gradients at start of each batch
       optim.zero_grad()

       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass
       outputs = model_13(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Backward pass and optimization
       loss.backward()
       optim.step()

       # Track and display training progress
       mean_training_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch}')
       loop.set_postfix(loss=loss.item())
   print("Mean Training Loss", np.mean(mean_training_loss))

   # Validation loop
   loop = tqdm(val_loader_2, leave=True)  # Progress bar for validation batches
   model_13.eval()  # Set model to evaluation mode
   mean_val_loss = []

   for batch in loop:
       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass (no gradients needed for validation)
       outputs = model_13(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Track and display validation progress
       mean_val_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch} Validation set')
       loop.set_postfix(loss=loss.item())
   print("Mean Validation Loss", np.mean(mean_val_loss))

  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 1.5257760535541889


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.1636637753450445


  0%|          | 0/14125 [00:00<?, ?it/s]

...

Mean Validation Loss 1.1825462786790781


  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 0.6769432402214909


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.3114761895804965


In [ ]:
# Save trained model parameters (weights & biases)
torch.save(model_13.state_dict(), "simple_distilbert_qa_data_13.model")

In [ ]:
# Initialize a new instance of our custom QA model
model_13 = SimpleQuestionDistilBERT(mod_13)

# Load previously saved model parameters from disk
model_13.load_state_dict(torch.load("simple_distilbert_qa_data_13.model"))

<ipython-input-11-d10db4231bf0>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_13.load_state_dict(torch.load("simple_distilbert_qa_data_13.model"))


<All keys matched successfully>

In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set(model_13, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [01:14<00:00, 47.26it/s]

Mean EM:  0.6428571428571429
Mean F-1:  0.7125609281280574


# Model_16
Train DistilBERT using SQuAD 2.0 --- data_2, Traditional Split; Alter training parameters to 4 epochs, dropout rate of 0.14 with AdamW optimizer rather than RMSProp

In [ ]:
# Load pre-trained DistilBERT model for masked language modeling
model_16 = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# Get model configuration (architecture, hyperparameters)
config_16 = DistilBertConfig.from_pretrained("distilbert-base-uncased")

# Extract base DistilBERT model without MLM head
mod_16 = model_16.distilbert

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_16 = SimpleQuestionDistilBERT(mod_16)
model_16.set_dropout_rate(0.14)
model_16.to(device)

# Verify the dropout rates for each layer
for name, module in model_16.named_modules():
    if isinstance(module, nn.Dropout):
        print(f"{name}: {module.p}")

set_dropout_rate
distilbert.embeddings.dropout: 0.14
distilbert.transformer.layer.0.attention.dropout: 0.14
distilbert.transformer.layer.0.ffn.dropout: 0.14
distilbert.transformer.layer.1.attention.dropout: 0.14
distilbert.transformer.layer.1.ffn.dropout: 0.14
distilbert.transformer.layer.2.attention.dropout: 0.14
distilbert.transformer.layer.2.ffn.dropout: 0.14
distilbert.transformer.layer.3.attention.dropout: 0.14
distilbert.transformer.layer.3.ffn.dropout: 0.14
distilbert.transformer.layer.4.attention.dropout: 0.14
distilbert.transformer.layer.4.ffn.dropout: 0.14
distilbert.transformer.layer.5.attention.dropout: 0.14
distilbert.transformer.layer.5.ffn.dropout: 0.14
dropout: 0.14


In [ ]:
# Set model to training mode (enables dropout, batch norm, etc.)
model_16.train()

# Initialize RMSprop optimizer and learning rate of 4e-5
optim = AdamW(model_16.parameters(), lr=4e-5)

In [ ]:
# Train the DistilBERT model with data_2 for 4 epochs
# Validation data used to evaluate performance during training

epochs = 4

for epoch in range(epochs):
   # Training loop
   loop = tqdm(loader_2, leave=True)  # Progress bar for training batches
   model_16.train()  # Set model to training mode
   mean_training_loss = []

   for batch in loop:
       # Zero gradients at start of each batch
       optim.zero_grad()

       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass
       outputs = model_16(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Backward pass and optimization
       loss.backward()
       optim.step()

       # Track and display training progress
       mean_training_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch}')
       loop.set_postfix(loss=loss.item())
   print("Mean Training Loss", np.mean(mean_training_loss))

   # Validation loop
   loop = tqdm(val_loader_2, leave=True)  # Progress bar for validation batches
   model_16.eval()  # Set model to evaluation mode
   mean_val_loss = []

   for batch in loop:
       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass (no gradients needed for validation)
       outputs = model_16(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Track and display validation progress
       mean_val_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch} Validation set')
       loop.set_postfix(loss=loss.item())
   print("Mean Validation Loss", np.mean(mean_val_loss))

  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 1.4852496564208935


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.1609763240824853


  0%|          | 0/14125 [00:00<?, ?it/s]

...

Mean Validation Loss 1.2520641732049573


  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 0.565563508329534


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.3189035853504403


In [ ]:
# Save trained model parameters (weights & biases)
torch.save(model_16.state_dict(), "simple_distilbert_qa_data_16.model")

In [ ]:
# Initialize a new instance of our custom QA model
model_16 = SimpleQuestionDistilBERT(mod_16)

# Load previously saved model parameters from disk
model_16.load_state_dict(torch.load("simple_distilbert_qa_data_16.model"))

<ipython-input-19-69432960d819>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_16.load_state_dict(torch.load("simple_distilbert_qa_data_16.model"))


<All keys matched successfully>

In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set(model_16, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [01:03<00:00, 55.05it/s]

Mean EM:  0.6352857142857142
Mean F-1:  0.7056830390810044
